In [1]:
import glob
import pandas as pd

In [2]:
alldata = pd.read_csv('combined_new.csv')

In [ ]:
def impute_missing(alldata):
    columns = alldata.columns
    index = alldata.index
    missing = SimpleImputer(missing_values='np.nan', strategy='mean')
    missing.fit(alldata)
    alldata= pd.DataFrame(missing.transform(alldata),columns=columns,index=index)
    return alldata

In [ ]:
def traintestSplit(alldata):
    alldata=impute_missing(alldata)
    Y=alldata["SepsisLabel"].astype('int')
    X=alldata.drop(["SepsisLabel", "PatientID"],axis=1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)
    print("Number of samples in train data: " + str(len(X_train)))
    print("Number of samples in validation data: " + str(len(X_val)))
    print("Number of samples in test data: " + str(len(X_test)))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [ ]:
def groupAverageForNone(): 
    tdata = pd.DataFrame()
    pdata = pd.DataFrame({"PatientID":["0"]})
    alldata=pd.read_csv("/training/output/combined.psv",sep="|")
    alldata = alldata.drop(alldata.columns[[0]], axis=1)
    attrcolumns = ["HR","O2Sat","Temp","SBP","MAP","DBP","Resp","EtCO2","BaseExcess","HCO3","FiO2","pH","PaCO2","SaO2","AST","BUN","Alkali  nephos","Calcium","Chloride","Creatinine","Bilirubin_direct","Glucose","Lactate","Magnesium","Phosphate","Potassium","Bilirubin_total","TroponinI","Hct","Hgb","PTT","WBC","Fibrinogen","Platelets"]
    print("data read")
    print(alldata.isna().sum())
    cols = alldata.columns
    for rowdata in alldata.itertuples():
        if (pdata.empty == True) | (pdata[pdata.PatientID == getattr(rowdata,"PatientID")].shape[0]==0):
            print(getattr(rowdata,"PatientID"))
            pdata = alldata[alldata.PatientID == getattr(rowdata,"PatientID")]
            unit1mean = pdata["Unit1"].mean()
            unit2mean = pdata["Unit2"].mean()
            if math.isnan(unit1mean):
                unit1mean = math.floor(alldata["Unit1"].mean())
            else:
                unit1mean = math.floor(unit1mean)
            if math.isnan(unit2mean):
                unit2mean = math.floor(alldata["Unit2"].mean())
            else:
                unit2mean = math.floor(unit2mean)         
            alldata.at[rowdata.Index,"Unit1"] = unit1mean
            alldata.at[rowdata.Index,"Unit2"] = unit2mean
            samegender = alldata[(alldata.Gender == getattr(rowdata,"Gender")) & (alldata.PatientID != getattr(rowdata,"PatientID"))]
            sameage = samegender[(samegender.Age == getattr(rowdata,"Age"))]
            if(sameage.empty == True):
                sameage = samegender[(samegender.Age >= getattr(rowdata,"Age")-2) & (samegender.Age <= getattr(rowdata,"Age")+2)]
            if(sameage.empty == True):
                sameage = samegender
            sameadmit = sameage[(sameage.HospAdmTime == getattr(rowdata,"HospAdmTime"))]
            if(sameadmit.empty == True):
                sameadmit = sameage[(sameage.HospAdmTime >= getattr(rowdata,"HospAdmTime") - 3) & (sameage.HospAdmTime <= getattr(rowdata,"HospAdmTime") + 3)]
            if(sameadmit.empty == True):
                sameadmit = sameage
            sameicu = sameadmit[(sameadmit.ICULOS == getattr(rowdata,"ICULOS"))]
            if(sameicu.empty == True):
                sameicu = sameadmit[(sameadmit.ICULOS >= getattr(rowdata,"ICULOS") - 2) & (sameadmit.ICULOS <= getattr(rowdata,"ICULOS") + 2)]
            if(sameicu.empty == True):
                sameicu = sameadmit[(sameadmit.ICULOS >= getattr(rowdata,"ICULOS") - 10) & (sameadmit.ICULOS <= getattr(rowdata,"ICULOS") + 10)]
            if(sameicu.empty == True):
                sameicu = sameadmit
            sameunit1 = sameicu[(sameicu.Unit1 == getattr(rowdata,"Unit1"))]
            if(sameunit1.empty == True):
                sameunit1 = sameicu
            sameunit2 = sameunit1[(sameunit1.Unit2 == getattr(rowdata,"Unit2"))]
            if(sameunit2.empty == True):
                sameunit2 = sameunit1
        for j in attrcolumns:
            jval = getattr(rowdata,j)
            if(math.isnan(jval)):
                meanval=pdata[j].mean()
                if(math.isnan(meanval)):
                    meanval = sameunit2[j].mean()
                    if(math.isnan(meanval)):
                        meanval = sameunit1[j].mean()
                    if(math.isnan(meanval)):
                        meanval = sameicu[j].mean()
                    if(math.isnan(meanval)):
                        meanval = sameadmit[j].mean()
                    if(math.isnan(meanval)):
                        meanval = sameage[j].mean()
                    if(math.isnan(meanval)):
                        meanval = samegender[j].mean()
                    if(math.isnan(meanval)):
                        meanval = 0  
                alldata.at[rowdata.Index,j] = meanval 
    tdata=alldata
    tdata.to_csv("/training/output/massaged.psv", sep='|',index=False)        
    return alldata

In [ ]:
def loadDatasetMassaged():
    alldata=pd.read_csv("/training/output/massaged.psv",sep="|")
    print("loaded massaged data")
    return alldata